A `Caller` is the basic structure that wraps all logic required for LLM call-and-response.

A `Caller` renders a conversation with a specific LLM client and call parameters (assumes OpenAI-compatibility through a framework like `aisuite`).
This allows every Caller instance to use a different model and/or parameters, and sets expectations for the Caller instance.

Additionally, `Callers` can be used as functions/tools in tool-calling workflows by leveraging `Caller.function_schema` which defines the inputs the Caller expects as a JSON schema.

Since a `Caller` has a specific client and model assigned, this effectively allows us to use Callers to route to specific models for specific use cases.
Since Callers can behave as functions themselves, we enable complex workflows where Callers can call Callers (ad infinitum ad nauseum).

In [ ]:
import json
import logging
import os
import re
from string import Template as StringTemplate
import textwrap
from typing import cast

from jinja2 import Template as JinjaTemplate
import json_repair
from pydantic import BaseModel, Field, ValidationError as PydanticValidationError, create_model

import aisuite
import openai

from yaaal.core.caller import Caller
from yaaal.core.handler import ResponseHandler, ToolHandler
from yaaal.core.validator import PassthroughValidator, PydanticValidator, RegexValidator, ToolValidator
from yaaal.types_.base import JSON
from yaaal.types_.core import Conversation, Message
from yaaal.utilities import basic_log_config, format_json

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
basic_log_config()
logging.getLogger("yaaal").setLevel(logging.DEBUG)
logger = logging.getLogger(__name__).setLevel(logging.DEBUG)

## Quick Start


In [ ]:
# all Callers require a client and a model to call.
# `yaaal` is built around OpenAI-compatible APIs primarily provided by `aisuite`
client = aisuite.Client(
    provider_configs={
        "openai": {"api_key": os.environ["YAAAL_OPENAI_API_KEY"]},
        "anthropic": {"api_key": os.environ["YAAAL_ANTHROPIC_API_KEY"]},
        # ...
    }
)
# `aisuite` specifies models in "provider:model" format
# model = "openai:gpt-4o-mini"
model = "anthropic:claude-3-5-haiku-latest"

In [ ]:
# A `ChatCaller`
chat_caller = Caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    description="chat",
    instruction="You are a helpful assistant",
    # input_template=StringTemplate("$input"),
    # input_params=create_model(
    #     "Chat",
    #     __doc__="a chatbot",
    #     input=(str, ...),
    # ),
)

2025-03-30 20:17:10,892 - DEBUG    - yaaal.core.caller - request_params:215 - All API requests for Caller will use params : {'temperature': 0.7}


A Caller's renders messages based on the instruction (used to create the System message), and the input_template (used to render the User message).

An instruction may be string, StringTemplate, or JinjaTemplate.
By default, the input_template is `None`, which corresponds to simply expecting `input` and passing it as a User message.

If a template is provided for instructions or input_template, input_params must be specified so that the Caller understands what the expected kwargs are.

Kwargs that are not `input` must be passed in the `state` dict.

In [ ]:
# Caller `function_schema` attribute is based on its Prompt signature
print(format_json(str(chat_caller.function_schema)))

{
  "description": "chat",
  "properties": {
    "input": {
      "title": "Input",
      "type": "string",
    },
  },
  "required": [
    "input",
  ],
  "title": "caller",
  "type": "object",
  "additionalProperties": false,
}


In [ ]:
# Render conversation using the new render method
print(chat_caller.render(input="Who is Harry Potter?"))

messages=[{
  "role": "system",
  "content": "You are a helpful assistant",
}, {
  "role": "user",
  "content": "Who is Harry Potter?",
}]


In [ ]:
# Callers are callable; treat them like functions
response = chat_caller(input="Who is Harry Potter?")

print(textwrap.fill(response, replace_whitespace=False))

Harry Potter is a fictional character created by British author J.K.
Rowling. He is the protagonist of the widely popular Harry Potter book
series, which consists of seven novels published between 1997 and
2007. In the story, Harry is a young wizard who discovers on his 11th
birthday that he is famous in the magical world for surviving an
attack by the evil dark wizard Lord Voldemort when he was an infant.
Key details about Harry Potter include:

1. Orphaned as a baby when
Voldemort killed his parents, Lily and James Potter
2. Raised by his
non-magical (Muggle) aunt and uncle, the Dursleys
3. Attends Hogwarts
School of Witchcraft and Wizardry
4. Belongs to Gryffindor House
5.
Known for his distinctive lightning bolt-shaped scar
6. Becomes a
central figure in the fight against Lord Voldemort
7. Best friends
with Ron Weasley and Hermione Granger

The book series follows Harry's
adventures from ages 11 to 17, chronicling his growth, magical
education, and ultimate confrontation with Volde

In [ ]:
# A `RegexCaller` validates the response with a regex pattern
pattern = re.compile(r"\b[A-E]\b(?!.*\b[A-E]\b)")

system_instructions = """
The following are multiple choice questions (with answers) about Star Wars.

What is the model designation of an X-Wing?
A. T-65B
B. BTL-A4
C. RZ-1
D. A/SF-01
Answer: A
"""

user_template = """
{{input}}
Answer:
""".strip()


class MCQAQuestion(BaseModel):
    """Multiple choice question."""

    input: str = Field(..., description="The multiple choice question")


regex_caller = Caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    description="Multiple-choice question answering",
    instruction=system_instructions,
    input_template=JinjaTemplate(user_template),
    input_params=MCQAQuestion,
    output_validator=pattern,
)

2025-03-30 20:22:04,777 - DEBUG    - yaaal.core.caller - request_params:215 - All API requests for Caller will use params : {'temperature': 0.7}


In [ ]:
print(format_json(str(regex_caller.function_schema)))

{
  "description": "Multiple choice question.",
  "properties": {
    "input": {
      "description": "The multiple choice question",
      "title": "Input",
      "type": "string",
    },
  },
  "required": [
    "input",
  ],
  "title": "MCQAQuestion",
  "type": "object",
  "additionalProperties": false,
}


In [ ]:
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

# Render conversation using the new render method
print(regex_caller.render(input=question))

messages=[{
  "role": "system",
  "content": "The following are multiple choice questions (with answers) about Star 
              Wars.             
             
             What is the model designation of an X-Wing?             
             A. T-65B             
             B. BTL-A4             
             C. RZ-1             
             D. A/SF-01             
             Answer: A",
}, {
  "role": "user",
  "content": "Han Solo is:             
             A. A scoundrel             
             B. A scruffy nerfherder             
             C. A smuggler             
             D. The owner of the Millennium Falcon             
             E. All of the above             
             Answer:",
}]


In [ ]:
response = regex_caller(input=question)
print(response)

if response == "E":
    print("Success! 🎉")
# print(textwrap.fill(response, replace_whitespace=False))

E
Success! 🎉


In [ ]:
# A `StructuredCaller` validates the response with a Pydantic model, and is good for structure data extraction
class Person(BaseModel, extra="ignore"):
    """A Person's characteristics."""

    name: str
    age: int
    favorite_color: str


# Notes on fstrings with jinja templates --
# The fstring allows us to substitute in variables before
# dynamically rendering the template in the Prompt.
# Because we're using fstrings, we have to double the `{}`
# so python understands they do not indicate an fstring substitution.
system_instructions = f"""
Identify facts about a person as they introduce themselves.

Respond in a format that matches the following json schema:

<schema>
{Person.model_json_schema()}
</schema>
""".strip()

structured_caller = Caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    description="person details",
    instruction=system_instructions,
    # input_template=JinjaTemplate("{{input}}"),
    # input_params=create_model(
    #     "structured_caller",
    #     __doc__="person details",
    #     input=(str, ...),
    # ),
    output_validator=Person,
)

2025-03-30 20:17:25,786 - DEBUG    - yaaal.core.caller - request_params:215 - All API requests for Caller will use params : {'temperature': 0.7, 'tools': [{'type': 'function', 'function': {'name': 'Person', 'strict': True, 'parameters': {'description': "A Person's characteristics.", 'properties': {'name': {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 'integer'}, 'favorite_color': {'title': 'Favorite Color', 'type': 'string'}}, 'required': ['name', 'age', 'favorite_color'], 'title': 'Person', 'type': 'object', 'additionalProperties': False}, 'description': "A Person's characteristics."}}], 'tool_choice': {'type': 'tool', 'name': 'Person'}}


In [ ]:
print(format_json(str(structured_caller.function_schema)))

{
  "description": "person details",
  "properties": {
    "input": {
      "title": "Input",
      "type": "string",
    },
  },
  "required": [
    "input",
  ],
  "title": "caller",
  "type": "object",
  "additionalProperties": false,
}


In [ ]:
introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

print(structured_caller.render(input=introduction))

messages=[{
  "role": "system",
  "content": "Identify facts about a person as they introduce themselves.             
             
             Respond in a format that matches the following json schema:             
             
             <schema>             
             {'description': "A Person's characteristics.", 'properties': {'name': 
              {'title': 'Name', 'type': 'string'}, 'age': {'title': 'Age', 'type': 
              'integer'}, 'favorite_color': {'title': 'Favorite Color', 'type': 
              'string'}}, 'required': ['name', 'age', 'favorite_color'], 'title': 
              'Person', 'type': 'object'}             
             </schema>",
}, {
  "role": "user",
  "content": "Hi, my name is Bob and I'm 42.  I work in a button factory, and my 
              favorite color is blue.",
}]


In [ ]:
response = structured_caller(input=introduction)

print(type(response))
print(format_json(response.model_dump()))

<class '__main__.Person'>
{
  "name": "Bob",
  "age": 42,
  "favorite_color": "blue",
}


Callers using Pydantic Handlers still return an AssistantMessage; it was validated internally before returning to the user.

This means we still have to re-validate if we want the response as a Pydantic model.


In [ ]:
# A `ToolCaller` can choose to call tools or respond like a normal LLM.
tool_caller = Caller(
    client=client,
    model=model,
    request_params={"temperature": 0.7},
    description="tool use",
    instruction="Use the best tool for the task.",
    # input_template="{{input}}",
    # input_params=create_model("ToolInput", input=(str, ...)),
    tools=[regex_caller, structured_caller],
    auto_invoke=True,
)

2025-03-30 20:24:47,768 - DEBUG    - yaaal.core.caller - request_params:215 - All API requests for Caller will use params : {'temperature': 0.7, 'tools': [{'type': 'function', 'function': {'name': 'MCQAQuestion', 'strict': True, 'parameters': {'description': 'Multiple choice question.', 'properties': {'input': {'description': 'The multiple choice question', 'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'MCQAQuestion', 'type': 'object', 'additionalProperties': False}, 'description': 'Multiple choice question.'}}, {'type': 'function', 'function': {'name': 'caller', 'strict': True, 'parameters': {'description': 'person details', 'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'caller', 'type': 'object', 'additionalProperties': False}, 'description': 'person details'}}], 'tool_choice': {'type': 'auto'}}


In [ ]:
print(format_json(str(tool_caller.function_schema)))

{
  "description": "tool use",
  "properties": {
    "input": {
      "title": "Input",
      "type": "string",
    },
  },
  "required": [
    "input",
  ],
  "title": "caller",
  "type": "object",
  "additionalProperties": false,
}


In [ ]:
# the tool_caller will automatically add the tools to the request parameters
print(format_json(tool_caller.request_params))

{
  "temperature": 0.7,
  "tools": [
    {
      "type": "function",
      "function": {
        "name": "MCQAQuestion",
        "strict": true,
        "parameters": {
          "description": "Multiple choice question.",
          "properties": {
            "input": {
              "description": "The multiple choice question",
              "title": "Input",
              "type": "string",
            },
          },
          "required": [
            "input",
          ],
          "title": "MCQAQuestion",
          "type": "object",
          "additionalProperties": false,
        },
        "description": "Multiple choice question.",
      },
    },
    {
      "type": "function",
      "function": {
        "name": "caller",
        "strict": true,
        "parameters": {
          "description": "person details",
          "properties": {
            "input": {
              "title": "Input",
              "type": "string",
            },
          },
          "required": [


In [ ]:
introduction = """
Hi, my name is Bob and I'm 42.  I work in a button factory, and my favorite color is blue.
""".strip()

print(tool_caller.render(input=introduction))

messages=[{
  "role": "system",
  "content": "Use the best tool for the task.",
}, {
  "role": "user",
  "content": "Hi, my name is Bob and I'm 42.  I work in a button factory, and my 
              favorite color is blue.",
}]


In [ ]:
# this should call the person schema tool
response = tool_caller(input=introduction)

print(type(response))
print(format_json(response.model_dump()))

2025-03-30 20:25:00,034 - DEBUG    - yaaal.core.handler - _invoke:187 - Invoking caller with params: input='Name: Bob, Age: 42, Occupation: Button Factory Worker, Favorite Color: Blue'


<class '__main__.Person'>
{
  "name": "Bob",
  "age": 42,
  "favorite_color": "Blue",
}


In [ ]:
# this should call the Star Wars QA tool
question = """
Han Solo is:
A. A scoundrel
B. A scruffy nerfherder
C. A smuggler
D. The owner of the Millennium Falcon
E. All of the above
""".strip()

response = tool_caller(input=question)

print(type(response))
print(format_json(response.model_dump() if hasattr(response, "model_dump") else response))

2025-03-30 20:25:08,674 - DEBUG    - yaaal.core.handler - _invoke:187 - Invoking MCQAQuestion with params: input='Han Solo is:\nA. A scoundrel\nB. A scruffy nerfherder\nC. A smuggler\nD. The owner of the Millennium Falcon\nE. All of the above'


In [ ]:
# this should pass through as a normal chat
response = tool_caller(input="Hello World!")

print(type(response))
print(format_json(response.model_dump() if hasattr(response, "model_dump") else response))

<class 'str'>
"Hello! How can I assist you today?"
